In [1]:
import scipy.sparse as sps
import numpy as np
import pandas as pd
import scipy as sc
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = np.genfromtxt('../input/train.csv' , delimiter=',',dtype=int)[1:]
test = np.genfromtxt('../input/target_playlists.csv', delimiter = ',', dtype=int)[1:]

playlist = data[:,0]
song = data[:,1]


In [3]:
unique_playlist = list(set(playlist))
unique_songs = list(set(song))

num_playlists = len(unique_playlist)
num_songs = len(unique_songs)


In [4]:
mat = sps.lil_matrix((num_playlists,num_songs))
mat[playlist,song] = 1
csr_mat = sc.sparse.csr_matrix(mat)


In [5]:
similarities = cosine_similarity(csr_mat, dense_output=False)

In [6]:
class TopPopRecommender(object):

    def fit(self, train):
        
        self.train = train

        itemPopularity = (train>0).sum(axis=0)
        itemPopularity = np.array(itemPopularity).squeeze()

        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popularItems = np.argsort(itemPopularity)
        self.popularItems = np.flip(self.popularItems, axis = 0)
    
    
    def recommend(self, user_id, at=10, remove_seen=True):

        if remove_seen:
            unseen_items_mask = np.in1d(self.popularItems, self.train[user_id].indices,
                                        assume_unique=True, invert = True)

            unseen_items = self.popularItems[unseen_items_mask]

            recommended_items = unseen_items[0:at]

        else:
            recommended_items = self.popularItems[0:at]
            
        recommended_items = " ".join(str(i) for i in recommended_items)
        return recommended_items

In [7]:
result = []
for elem in test:
    target_row = elem

    row_start = similarities.indptr[target_row]
    row_end = similarities.indptr[target_row+1]
    similarity_vec = similarities.data[row_start:row_end]
    similarity_vec = np.argsort(similarity_vec)
    similarity_vec = np.flip(similarity_vec,0)
    similarity_vec = similarity_vec[:100]
    similar_playlist = similarities.indices[similarity_vec]

    training_set = csr_mat[similar_playlist,:]
    
    top_recommender = TopPopRecommender()
    top_recommender.fit(training_set)
    recommendation = top_recommender.recommend(elem)
    temp = [elem,recommendation]
    result.append(temp)
    
rec = pd.DataFrame(result)
rec.to_csv("sample_submission.csv", index = False, header = ["playlist_id", "track_ids"])

IndexError: index (106) out of range